In [ ]:
import numpy as np
import random
import math
import json
np.seterr(divide='ignore', invalid='ignore')

matrixs = np.load('Our.npz')
V = np.load('all_2939D.npy')
user_category = np.load('user_category_1489.npy')
YouTuber_category = np.load('YouTuber_category_0.7.npy')
user_following = np.load('user_following_1489.npy')
with open('ytList.json', 'r') as openfile: 
    ytList = json.load(openfile) 
list(matrixs.keys())
U = matrixs['U']
Y = matrixs['Y']
A = matrixs['A']
E = matrixs['E']
Wu = matrixs['Wu']
Wy = matrixs['Wy']
Wa = matrixs['Wa']
Wv = matrixs['Wv']
B = matrixs['B']
# alpha = Rugi(WuU+WyY+WaA+WvEV)

#over5 = 0
#for num in YouTuber_followers:
#    if num >= 5:
#        over5+=1
#print('The num of followers over 5:',over5)
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
#print('user_category after normalized by max...')
#print('user_category_norm shape ',user_category_norm.shape)
#print('user cateogory norm',user_category_norm)
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)
test_amount = 150
yt_test_amount = 18
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(5)
test_idx = sorted(random.sample(user_idx,test_amount))
#print(test_idx)
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        #print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        #print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
print(avg)

In [ ]:
final_map = np.zeros(shape=(len(test_idx),88))
map_id = 0
for idu in test_idx:
    alpha = np.dot(Wu[idu],U[idu].T)
    added_alpha = np.add(np.maximum(alpha, 0),0.0000000001)
    #print(added_alpha/np.sum(added_alpha))
    positive_alpha = []
    positive_yts = []
    for i in range(88):
        r =np.max(YouTuber_category[i]*user_category_norm[idu])
        positive_alpha.append(r*added_alpha[i])
        positive_yts.append(ytList[i])
    #print(positive_alpha)
    norm_alpha = positive_alpha/np.sum(positive_alpha)
    final_map[map_id] = norm_alpha
    map_id+=1
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(final_map);
plt.colorbar()
plt.show()

In [ ]:
selected_idy = [78,14, 24, 57]+[66,40, 10, 74]
selected_idx = [0,3,18,91,112,113]
selected_map = np.zeros(shape=(len(selected_idx),len(selected_idy)))
for i in range(len(selected_idx)):
    for j in range(len(selected_idy)):
        selected_map[i][j]= final_map[selected_idx[i]][selected_idy[j]]

In [ ]:
axis_ytList = []
for youtuber_idy in selected_idy:
    #print(ytList[youtuber_idy],list(YouTuber_category[youtuber_idy]).index(np.max(YouTuber_category[youtuber_idy])))
    axis_ytList.append(ytList[youtuber_idy])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#plt.xlim([0,8])  # x軸邊界
#plt.ylim([0,6])  # y軸邊界
plt.xticks(np.arange(len(axis_ytList)),axis_ytList,rotation=90)
plt.yticks(np.arange(0,6),[str(val) for val in selected_idx])
plt.imshow(selected_map)
#plt.yticks(np.arange(-1, 7, step=1))
plt.xlabel('YouTuber')
plt.ylabel('User ID')
#plt.title('Examples of attention')
plt.colorbar()
plt.show()